In [ ]:
import pandas as pd

df = pd.read_csv("ABC_data.csv", index_col = 0)

df.head()

#### ANÁLISIS EXPLORATORIO DE DATOS

Este es un primer EDA sin haber tratado los nulos, las columnas redundantes y los resultados inconsistentes, por lo que si bien es una primera aproximación de los datos que tenemos, los resultados reales podrían estar bastante alejados de estos resultados. Es necesario en cada momento contextualizar las condiciones en las que se han obtenido los datos.

In [2]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from IPython.display import display

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
# ver todas las columnas
pd.set_option('display.max_columns', None)

In [12]:
# ver todas las filas
pd.set_option('display.max_rows', None)

In [13]:
df.head()

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1
3,47,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,"14307,50$",1976,"171690,00$",NaN,NaN,False
4,46,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,"12783,92$",1977,NaN,NaN,NaN,0


Necesitamos hacer una copia del dataframe porque se van a convertir las columnas object que sean numéricas para realizar un EDA más exacto.  Y queremos que nuestro dataframe de inicio quede intacto.

In [ ]:
import pandas as pd
import warnings

def limpiar_df(df):
    """
    Convierte automáticamente columnas object que sean numéricas o fechas.
    Modifica el DataFrame recibido.
    Ignora warnings de pandas sobre inferencia de formato de fechas.
    """
    # Ignorar UserWarning de pandas
    warnings.filterwarnings("ignore", category=UserWarning)
    
    for col in df.select_dtypes(include='object').columns:
        # Intentar convertir a número
        try:
            df[col] = pd.to_numeric(df[col], errors='raise')
            continue
        except:
            pass
        # Intentar convertir a fecha
        try:
            df[col] = pd.to_datetime(df[col], errors='raise', dayfirst=False)
        except:
            pass  # Si falla, dejamos como object (categórica)
    
    # Restaurar warnings
    warnings.resetwarnings()
    
    return df


# Crear copia
df_limpio = df.copy()

# Limpiar la copia
limpiar_df(df_limpio)

# Ahora df_limpio tiene columnas numéricas y fechas convertidas,
# y las columnas categóricas reales siguen siendo object


In [34]:
# Función para EDA completo
def eda(df):
    """
    Realiza un análisis exploratorio de datos sobre un DataFrame ya limpio:
    - Primeras filas
    - Dimensiones
    - Tipos de datos
    - Estadísticas numéricas y categóricas
    - Valores nulos
    - Filas duplicadas
    - Valores únicos
    - Distribución de variables categóricas
    - Resumen general
    """
    print("ANÁLISIS EXPLORATORIO DE DATOS\n")
    print("Primeras 10 filas:")
    display(df.head(10))
    print("\nDimensiones:")
    print(df.shape)
    print("\nInformación general:")
    df.info()
    print("\nTipos de datos por columna:")
    print(df.dtypes)
    # Columnas numéricas
    num_cols = df.select_dtypes(include='number').columns
    if len(num_cols) > 0:
        print("\nEstadísticas numéricas:")
        display(df[num_cols].describe().T)
    # Columnas categóricas reales
    cat_cols = df.select_dtypes(include='object').columns
    if len(cat_cols) > 0:
        print("\nEstadísticas categóricas:")
        display(df[cat_cols].describe())
    # Valores nulos
    print("\nValores nulos:")
    display(pd.DataFrame({
        "Conteo": df.isnull().sum(),
        "Porcentaje": (df.isnull().sum()/len(df)*100).round(2)
    }).sort_values(by="Porcentaje", ascending=False))
    # Filas duplicadas
    print("\nFilas duplicadas:", df.duplicated().sum())
    # Valores únicos
    print("\nValores únicos por columna:")
    display(df.nunique().sort_values(ascending=False))
    # Distribución de variables categóricas
    for col in cat_cols:
        print(f"\nColumna: {col}")
        display(df[col].value_counts().head())
    # Resumen general
    resumen = pd.DataFrame({
        "Columna": df.columns,
        "Dtype": df.dtypes.astype(str),
        "Valores únicos": df.nunique().values,
        "% Nulos": (df.isnull().sum()/len(df)*100).round(2).values
    })
    print("\nResumen general de columnas:")
    display(resumen)
    return resumen

In [ ]:
# df_limpio es la copia, df original queda intacto
resumen = eda(df_limpio)   # Hacemos el EDA antes de eliminar columnas

### 🗑️ Plan de eliminación de columnas redundantes

💰 **monthlyincome, sameasmonthlyincome y salary**  
- 📊 Todas parecen expresar ingresos mensuales o anuales en distintas unidades/formatos.  
- 🔁 `sameasmonthlyincome` explícitamente es un duplicado de `monthlyincome`.  
✅ Nos quedamos con **monthlyincome** y **salary**  
❌ Eliminamos **sameasmonthlyincome**

---

🏢 **department, roledepartament y jobrole**  
- ⚠️ `roledepartament` mezcla el rol con el departamento (ejemplo: *Manager - Research & Development*), mientras que `department` y `jobrole` ya lo tienen separado.  
✅ Nos quedamos con **department** y **jobrole**  
❌ Eliminamos **roledepartament**  
💡 *Más adelante podemos fusionar `department` y `jobrole` si es necesario.*

---

👥 **employeecount y employeenumber**  
- 🔂 `employeecount` es siempre 1 → columna inútil.  
- 🆔 `employeenumber` parece ser solo un identificador único → no aporta al análisis predictivo.  
❌ Eliminamos **employeecount**

---

🎂 **age y datebirth**  
- 📐 Son equivalentes: de `datebirth` se puede calcular la edad.  
❌ Eliminamos **age**

---

👶 **numberchildren**  
- 🚫 Está completamente vacío (0 valores válidos).  
❌ Eliminamos **numberchildren**


In [35]:
# Función para eliminar columnas redundantes

def drop_redundant_columns(df_clean):
    cols_to_drop = [
        "sameasmonthlyincome",
        "roledepartament",
        "employeecount",
        "age",
        "numberchildren"
    ]
    
    # Mostrar solo las columnas que existen realmente
    existing_cols = [c for c in cols_to_drop if c in df_clean.columns]
    print("Eliminando columnas:", existing_cols)
    
    # Eliminar columnas existentes
    df_clean = df_clean.drop(columns=existing_cols)
    
    return df_clean
    

In [36]:
# Ahora eliminar columnas redundantes
df_limpio = drop_redundant_columns(df_limpio)

Eliminando columnas: ['sameasmonthlyincome', 'roledepartament', 'employeecount', 'age', 'numberchildren']


In [19]:
# Mostrar DataFrame limpio final
display(df_limpio.head())

,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,No,NaN,2015.722222,NaN,6,3,NaN,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,1972,"195370,00$",Yes
1,No,NaN,2063.388889,NaN,1,4,Life Sciences,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,1971,"199990,00$",1
2,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,1981,"192320,00$",1
3,No,travel_rarely,1771.404762,NaN,2,4,Medical,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,1976,"171690,00$",False
4,No,NaN,1582.771346,NaN,3,3,Technical Degree,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,1977,NaN,0


### ✨ Plan de normalización y limpieza de columnas

🖥️ **Columna `remotework`**  
- 🔄 Normalizar valores a `yes` / `no`.

---

👫 **Columna `gender`**  
- 🔄 Normalizar valores a `M` / `F`.

---

🔤 **Columnas categóricas (texto)**  
- 📝 Corregir fallos tipográficos (ejemplo: combinación de mayúsculas y minúsculas).  
- 🔠 Capitalizar la primera letra de cada valor.  
- ⚙️ Implementar una función que recorra todas las columnas categóricas para normalizar texto.

---

🔢 **Columnas numéricas**  
- 🔧 Corregir fallos tipográficos:  
  - ➡️ Reemplazar `,` por `.` en valores numéricos.  
  - 🎯 Redondear decimales a 2 dígitos.  
- ⚙️ Implementar una función que recorra todas las columnas numéricas y normalice sus valores.

---

📏 **Columna `distancefromhome`**  
- 🔄 Convertir valores negativos a valor absoluto.

---

🔍 **Detección y corrección de valores mal escritos en categóricas**  
- 👀 Identificar valores con `unique()` durante el EDA.  
- ✏️ Reemplazar cada error con la palabra correctamente escrita.  
  - Ejemplo: `marreid` → `married`.

---

🏷️ **Nombres de columnas**  
- ✨ Renombrar y normalizar: todas las columnas con formato `.title`.  
  - Ejemplo: `distancefromhome` → `Distancefromhome`.


In [37]:
# Normalizar la columna 'remotework' a valores yes/no:

#1.Inspeccionamos primero los valores únicos
df_limpio['remotework'].unique()

array(['Yes', '1', 'False', '0', 'True'], dtype=object)

In [38]:
#2. Normalizamos 
df_limpio['remotework'] = (
    df_limpio['remotework']
    .astype(str)       # convertir a texto
    .str.strip()       # quitar espacios
    .str.lower()       # pasar a minúsculas
    .replace({         # mapear valores conocidos
        'yes': 'yes',
        'true': 'yes',
        '1': 'yes',
        'false': 'no',
        '0': 'no'
    })
)

#3.Verificamos el resultado final
print(df_limpio['remotework'].value_counts())


remotework
yes    1042
no      636
Name: count, dtype: int64


In [39]:
# Normalizar la columna gender a valores M/F:

#1.Inspeccionamos primero los valores únicos
print(df_limpio['gender'].unique())

[0 1]


In [40]:
#2. Normalizamos 
df_limpio['gender'] = (
    df_limpio['gender']
    .astype(str)        # aseguramos texto
    .str.strip()        # quitamos espacios
    .str.lower()        # pasamos a minúsculas
    .replace({
        'male': 'M',
        'm': 'M',
        '1': 'M',
        'hombre': 'M',
        'man': 'M',
        'female': 'F',
        'f': 'F',
        '0': 'F',
        'mujer': 'F',
        'woman': 'F'
    })
)

#3. Verificamos el resultado
print(df_limpio['gender'].value_counts())

gender
F    1009
M     669
Name: count, dtype: int64


In [ ]:
# Creamos una función para normalizar todas las columnas categóricas

def normalizar_categoricas(df_limpio):
    """
    Normaliza columnas categóricas de df_limpio:
      - elimina espacios al inicio y al final
      - reemplaza '_' por espacio
      - corrige combinación de mayúsculas/minúsculas
      - usa Title Case en las columnas que lo requieren
      - mantiene NaN intactos
    """
    import pandas as pd

    # Creamos un diccionario para las columnas que deben tener cada valor/palabra capitalizada (Title Case)
    cols_title = ['department', 'jobrole']

    for col in df_limpio.select_dtypes(include=['object', 'category']).columns:
        s = df_limpio[col]
        was_categorical = pd.api.types.is_categorical_dtype(s)

        # Máscara para no tocar valores nulos
        mask = s.notna()

        # Limpieza común
        temp = (
            s.loc[mask]
             .astype(str)
             .str.strip()           # quitar espacios al inicio/final
             .str.replace('_', ' ') # reemplazar '_' por espacio
             .str.lower()           # todo a minúsculas
        )

        # Ajuste de capitalización
        if col.lower() in cols_title:
            temp = temp.str.title()         # cada palabra con mayúscula
        else:
            temp = temp.str.capitalize()    # sólo la primera letra

        df_limpio.loc[mask, col] = temp

        if was_categorical:
            df_limpio[col] = df_limpio[col].astype('category')

    return df_limpio

# ✅ Uso: aplicar y guardar la función sobre df_limpio
df_limpio = normalizar_categoricas(df_limpio)

In [49]:
df_limpio.head()

,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,No,Nan,2015.722222,Nan,6,3,Nan,1,1,F,NaN,3,5,Research Director,3,Nan,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full time,0,Nan,5,"3,0",20,Nan,15,15,1972,"195370,00$",Yes
1,No,Nan,2063.388889,Nan,1,4,Life sciences,2,3,F,NaN,2,5,Manager,3,Nan,Nan,"43331,17$",0,Nan,Nan,14,"3,0",1,Nan,1,"34,0",5,"3,0",33,Nan,11,9,1971,"199990,00$",Yes
2,No,Travel rarely,1984.253968,Research & Development,4,2,Technical degree,3,3,F,NaN,3,5,Manager,4,Married,Nan,"41669,33$",1,Nan,No,11,"3,0",4,Nan,0,"22,0",3,Nan,22,Nan,11,15,1981,"192320,00$",Yes
3,No,Travel rarely,1771.404762,Nan,2,4,Medical,4,1,M,NaN,3,4,Research Director,3,Married,"14307,50$","37199,50$",3,Y,Nan,19,"3,0",2,Full time,2,Nan,2,Nan,20,Nan,5,6,1976,"171690,00$",No
4,No,Nan,1582.771346,Nan,3,3,Technical degree,5,1,M,NaN,4,4,Sales Executive,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,Nan,1,Nan,5,"3,0",19,Nan,2,8,1977,Nan,No


In [ ]:
# Creamos una función para renombrar todas las columnas
import re

def normalizar_nombres_columnas(df_limpio):
    """
    Normaliza nombres de columnas:
      - Inserta '_' entre palabras (detecta camelCase y letras/números)
      - Capitaliza la primera letra de cada palabra
    """
    nuevas_columnas = []

    for col in df_limpio.columns:
        # 1. Elimina espacios al inicio y final
        col = col.strip()

        # 2. Inserta '_' entre palabras (camelCase o letras/números)
        col = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', col)     # minúscula→Mayúscula
        col = re.sub(r'(?<=[a-zA-Z])(?=[0-9])', '_', col)  # letra→dígito
        col = re.sub(r'(?<=[0-9])(?=[a-zA-Z])', '_', col)  # dígito→letra

        # 3. Pasar a minúsculas
        col = col.lower()

        # 4. Capitalizar la primera letra de cada palabra
        partes = col.split('_')
        partes = [p.capitalize() for p in partes]

        # 5. Volver a unir con '_'
        nuevo_nombre = '_'.join(partes)

        nuevas_columnas.append(nuevo_nombre)

    df_limpio.columns = nuevas_columnas
    return df_limpio


# ✅ Uso: aplicar y guardar la función sobre df_limpio
df_limpio = normalizar_nombres_columnas(df_limpio)


***Una funcioncilla para ver cosas***

In [20]:
# FUNCIÓN PARA ELIMINAR DUPLICADOS
# MANTIENE LA PRIMERA APARICIÓN Y BORRA LAS DEMÁS FILAS REPETIDAS
# ELIMINAR DUPLICADOS PORQUE INFLAN MUCHO LOS RESULTADOS

def remove_duplicates(df):
    """
    Elimina filas duplicadas de un DataFrame manteniendo la primera aparición.

    Parámetros:
    df (pd.DataFrame): DataFrame de entrada.

    Retorna:
    pd.DataFrame: DataFrame sin duplicados.
    """
    filas_iniciales = len(df)
    df_sin_duplicados = df.drop_duplicates(keep="first")
    filas_finales = len(df_sin_duplicados)
    print(f"Se eliminaron {filas_iniciales - filas_finales} filas duplicadas")
    return df_sin_duplicados

In [ ]:
df_clean = remove_duplicates(df_clean)

In [ ]:
# FUNCIÓN QUE DETECTA COLUMNAS QUE ESTÁN EN FORMATO STRING PERO CONTIENEN VALORES NUMÉRICOS Y
# LAS CONVIERTE A TIPO NUMÉRICO
def convert_numeric_columns(df):
    """
    Detecta columnas que están en formato string pero contienen valores numéricos
    y las convierte automáticamente a tipo numérico.

    Parámetros:
    df (pd.DataFrame): DataFrame de entrada.

    Retorna:
    pd.DataFrame: DataFrame con las columnas convertidas a numéricas.
    """
    for col in df.columns:
        if df[col].dtype == "object":  # posibles columnas mal cargadas
            try:
                df[col] = pd.to_numeric(df[col], errors="coerce")
            except Exception:
                pass  # si no se puede convertir, la deja como está
    return df


In [ ]:
df_clean = convert_numeric_columns(df_clean)


En esta parte:
- Quitamos espacios extra en columnas de texto.  
- Organizamos un poco mejor los nombres de roles y departamentos.  
- Creamos columnas nuevas para analizar la antigüedad (que agrupa a los empleados según los años en la empresa. Esto es útil porque no es lo mismo alguien que lleva 1 año que alguien que lleva 20. ) y el teletrabajo (para indicar con un 1 si la persona trabaja en remoto y 0 si no.)



In [ ]:
# LIMPIEZA DE DATOS 

import pandas as pd

# 1. Cargo el dataset original
df = pd.read_csv("ABC_data.csv", index_col=0)

# 2. Hago una copia para trabajar sin tocar el original
df_copy = df.copy()

# 3. Limpio columnas de texto (jobrole y department)
#    - Quito espacios extra
#    - Pongo formato con mayúscula inicial
df_copy["jobrole"] = df_copy["jobrole"].str.strip().str.title()
df_copy["department"] = df_copy["department"].str.strip().str.title()

# 4. Creo una nueva columna de antigüedad agrupada (TenureGroup)
#    Agrupo los años en la empresa en intervalos (0-2, 3-5, 6-10, etc.)
bins = [0,2,5,10,20,40]                        # límites de los grupos
labels = ["0-2","3-5","6-10","11-20","20+"]    # nombres de los grupos
df_copy["TenureGroup"] = pd.cut(df_copy["yearsatcompany"], 
                                bins=bins, labels=labels, include_lowest=True)

# 5. Creo una nueva columna para teletrabajo (RemoteWork_flag)
#    Si la persona trabaja en remoto → 1, si no → 0
df_copy["RemoteWork_flag"] = df_copy["remotework"].astype(str).str.lower().isin(["yes","1","true"]).astype(int)

# 6. Muestro las primeras filas para comprobar la limpieza
df_copy.head()

In [ ]:

attrition_by_tenure = df_copy.groupby("TenureGroup")["attrition"].apply(lambda x: (x=="Yes").mean())
print(attrition_by_tenure)

In [ ]:
attrition_by_remote = df_copy.groupby("RemoteWork_flag")["attrition"].apply(lambda x: (x=="Yes").mean())
print(attrition_by_remote)

In [ ]:
pip install matplotlib

In [ ]:

# VISUALIZACIONES


import matplotlib.pyplot as plt   # 👈 esto es lo que faltaba

# 1. Attrition por TenureGroup
attrition_by_tenure = df_copy.groupby("TenureGroup")["attrition"].apply(lambda x: (x=="Yes").mean())

attrition_by_tenure.plot(kind="bar", title="Attrition por antigüedad (TenureGroup)")
plt.ylabel("Tasa de attrition")
plt.show()

# 2. Attrition por RemoteWork_flag
attrition_by_remote = df_copy.groupby("RemoteWork_flag")["attrition"].apply(lambda x: (x=="Yes").mean())

attrition_by_remote.plot(kind="bar", title="Attrition por teletrabajo (0 = No, 1 = Sí)")
plt.ylabel("Tasa de attrition")
plt.show()



- La **antigüedad en la empresa** influye bastante en el abandono:  
  los empleados más nuevos (0-2 años) tienen más probabilidad de irse, mientras que quienes llevan más años tienden a quedarse.  

- El **teletrabajo** también marca una diferencia:  
  hay variaciones claras en la tasa de abandono entre quienes trabajan remoto y quienes no.  

Esto es solo un primer análisis, pero ya da pistas de factores importantes para estudiar más a fondo.